## <span style="color:red">Metaprogramming in Python</span>

Con questo termine (<em>metaprogramming</em>) si intende la possibilità che un programma ha di "manipolare" (cioè cambiare, adattare) se stesso a tempo di esecuzione. 
Abbiamo già visto qualcosa in tal senso, ma in questo notebook vogliamo mettere meglio a fuoco il concetto

In [ ]:
def a_method(self,*args,**kwargs):
    '''Una funzione, che potrebbe diventare un metodo di una qualche classe...'''
    print("A do-nothing implementation")
    return True

#### Un primo esempio, utilizzando solo meccanismi già presi in considerazione

In [ ]:
def customize(cls):
    '''Customization function'''
    if include:
        cls.another_method = a_method
    return cls

In [ ]:
# La seguente porzione di codice è il più semplice esempio di possibili situazioni
# in cui le funzionalità complete (in questo caso, come vedremo) di una classe
# devono o possono solo essere decise a tempo di esecuzione, in quanto dipendenti
# da informazioni fornite dall'utente (o presenti in file di configurazione).

answer = input("The class must contain a_method? (Y/n): ")
if not answer or answer.upper()=='Y':
    include = True
else:
    include = False

In [ ]:
# Alla luce di quanto presentato sopra potremmo ad esempio scrivere
@customize
class a_class:
    pass

In [ ]:
a = a_class()
a.another_method()

### Verso le metaclassi

In Python3 (come più volte ricordato) è stata portata a completezza l'unificazione dei tipi definiti da utente e dei "tipi primitivi"

1. Ogni oggetto è istanza di una qualche classe
2. Ogni entità manipolabile nel linguaggio è un oggetto
3. Quindi anche le classi sono oggetti
4. Ne consegue che, per ogni oggetto x, sarà sempre vero che type(x)==x.\_\_class\_\_ (laddove la seconda scrittura si può impiegare)

In [ ]:
type(int)==int.__class__

In [ ]:
class foo:
    pass
type(foo) == foo.__class__

In [ ]:
x = foo()
type(x) == x.__class__

In [ ]:
type((3,3))

In [ ]:
type({'A':1, 'B':2}) == {'A':1, 'B':2}.__class__

In [ ]:
# Ma...
type(3) == 3.__class__

In [ ]:
# Se x è istanza di una class X, allora è chiaro che ...
type(3)

__Attenzione__: la "forma" del precedente output è dovuta a ipython (cioè, al fatto che ipython "intercetta" l'output dell'interprete python e, talvolta, lo elabora). Se "by-passiamo" ipython mediante stampa dell'output, vediamo il risultato che ci darebbe l'interprete

In [ ]:
print(type(3))

Dato che anche le classi sono oggetti, ci chiediamo ora quale sia il <u>tipo di una classe</u>

In [ ]:
print(type(foo))

__type__ è il tipo di ogni classe (attenzione anche qui a non confondere il concetto di tipo di una classe con quello di superclassi di una classe)

type è (naturalmente) anch'esso una classe il cui tipo è type stesso

In [ ]:
type(type)

__Una logica deduzione__. Se:

1. type è una classe
2. le istanze di type sono classi

allora la scrittura 

type() 

dovrebbe essere consentita e, come risultato, produrre una classe! 

E in effetti lo è, ma quale classe produce? Con quale nome?

Per rispondere in modo semplice, possiamo dire che type (in quanto classe) possiederà un metodo un metodo __init__; questo accetta tre parametri: nome della classe, superclassi e un dizionario che "confluirà" poi nel \_\_dict\_\_ della classe 

#### Esempio 1

In questo esempio viene istanziata la classe _type_ in modo che essa produca una classe, di nome _X_, con due attributi di classe, _A_ e _B_. 

In [ ]:
d = {'A':1, 'B':2}
T1 = type('X',(),d)

La due precedenti righe sono equivalenti alla seguente definizione

In [ ]:
class X:
    A = 1
    B = 1

La differenza sostanziale è che questa seconda è "statica", deve cioè essere scritta nel codice de programma. La prima invece è dinamica: sia il nome sia il dizionario possono essere il risultato di computazioni (quindi determinati a run-time).
Una differenza, peraltro di poco rilievo, è che per istanziare oggetti di tipo _X_ nel primo caso si deve scrivere
```python
x = T1()
```
mentre nel caso statico si usa la ben nota scrittura
```python
x = X()
```

Qualche verifica...

In [ ]:
T1.__dict__

In [ ]:
x = T1()

In [ ]:
x.A

In [ ]:
y = T1()

In [ ]:
y.A

In [ ]:
type(x)

In [ ]:
T1.A = 2

In [ ]:
x.A

In [ ]:
y.A

#### Esempio 2
Nell'esempio precedente _A_ e _B_ sono attributi di classe, come si evince anche dalle ultime "verifiche". Se vogliamo che ogni oggetto di _X_ abbia proprie variabili dinamiche (e vogliamo farlo secondo le regole di stile di Python) dobbiamo realizzare qualcosa di equivalente a:

In [ ]:
class X:
    def __init__(self,a,b):
        self.A = a
        self.B = b

Dobbiamo quindi inserire una funzione \_\_init\_\_  nel dizionario iniziale di _X_

In [ ]:
def myinit(self,a,b):
    self.A = a
    self.B = b

T2 = type('X', (), {'__init__': myinit})

In [ ]:
x = T2(1,2)

In [ ]:
x.A

In [ ]:
y = T2(3,4)

In [ ]:
print(x.A)
print(y.A)

#### Esempio 3

Vogliamo ora utilizzare anche il secondo parametro id _type_. Questo primo esempio è molto semplice

In [ ]:
class A:
    def __init__(self,x):
        self.x = x

Z = type('sum',(A,),{'y':1})

In [ ]:
print(A.__bases__) # __bases__ è un attributo il cui valore è la tupla con le superclassi
print(Z.y)
print(Z.__bases__)  

#### Esempio 4

Questo è un caso leggermente più complesso che però coincide (usando il modello "dinamico") con qualcosa che abbiamo già visto; pricisamente la definizione degli archi (<em>edge</em>) di un grafo pesato. A suo tempo scrivemmo:

In [ ]:
class edge(tuple):
    def __new__(cls,t,w=0):
        self = super().__new__(cls,t)
        self.w = w
        return self

In [ ]:
e = edge((1,2),3)

In [ ]:
print(f"Il peso dell'arco {e} è {e.w}")

Passiamo al contesto dinamico.

In [ ]:
def mynew(cls,t,w=0):
    self = cls.__bases__[0].__new__(cls,t) #cls.__bases__[0] è la prima superclasse di cls
    self.w = w
    return self

In [ ]:
T3 = type('edge',(tuple,),dict(__new__= mynew))

In [ ]:
T3.__bases__[0]

In [ ]:
z = T3((1,2),2)

In [ ]:
print(f"Il peso dell'arco {z} è {z.w}")

In [ ]:
type(z)

## Metaclassi

Detto in parole semplici, le metaclassi sono il meccanismo mediante il quale è possibile "customizzare" l'istanziazione di una classe.

Una classe è un'istanza di type che, abbiamo visto, è istanza di se stessa. Dunque type è una __metaclasse__ e sono metaclassi tutte le classi che ereditano da type

In [1]:
class una_metaclasse(type):
    def __init__(cls, name, bases, dictionary):
        '''Inizializzazione che esegue solo alcune stampe, che ci permettono di capire
           quando viene eseguito il codice e i parametri passati'''
        print(cls)
        print(name)
        print(bases)
        print(dictionary)
    foo = 0 #Ci servirà dopo....

Possiamo fare in modo che il tipo di una classe sia una metaclasse diversa da type utilizzando la seguente sintassi

```python
class NOMECLASSE(ELENCO_SUPERCLASSI,metaclass=NOMEMETACLASSE):
    CORPODELLACLASSE
```

Esempio

In [2]:
class S:
    foo = 10

class X(S,metaclass=una_metaclasse):
    pass

<class '__main__.X'>
X
(<class '__main__.S'>,)
{'__module__': '__main__', '__qualname__': 'X'}


Che cosa è successo?

Quando viene creata una classe (in questo caso X) il cui tipo è una metaclasse diversa da type:

1. vengono chiamati i metodi \_\_new\_\_ e \_\_init\_\_ della metaclasse
2. poichè (naturalmente) valgono i meccanismi di ereditarietà e la metaclasse eredita da type, se la metaclasse non definisce uno di tali metodi allora viene chiamato il metodo corrispondente di type 

Nel nostro caso _una_metaclasse_ definisce \_\_init\_\_ ma non \_\_new\_\_ quindi è stato prima chiamato il metodo \_\_new\_\_ di type e poi il metodo \_\_init\_\_ di una_metaclasse

Naturalmente possiamo anche scrivere il metodo \_\_new\_\_

In [ ]:
class una_metaclasse(type):
    def __new__(cls, name, bases, dictionary):
        '''In questa versione "intercettiamo" __new__ e chiamiamo esplicitamente
           la __new__ di type'''
        print(cls)
        print(name)
        print(bases)
        print(dictionary)
        return type.__new__(cls, name, bases, dictionary)
    foo = 10
        
class X(S, metaclass=una_metaclasse):
    '''X eredita da tuple ma il suo tipo è una_metaclasse'''
    foo = 5
    pass

Qualche esercizio per capire se abbiamo capito...

In [ ]:
x = X()
# Quali saranno i valori stampati dai seguenti comandi?
print(x.foo)
print(type(x).foo)
print(type(X).foo)
print(type(type(x)).foo)

#### Ritorniamo ora all'esempio iniziale di "customizzazione"

In [ ]:
def a_method(self,*args,**kwargs):
    '''Una funzione, che potrebbe diventare un metodo di una qualche classe...'''
    print("A do-nothing implementation")
    return True

In [ ]:
class customize(type):
    def __init__(cls, clsname, superclasses, attributedict):
        answer = input("The class must contain a_method? (Y/n): ")
        if not answer or answer.upper()=='Y':
            include = True
        else:
            include = False
        if include:
            cls.another_method = a_method

In [ ]:
class a_class(metaclass=customize):
    pass

In [ ]:
an_object = a_class()

In [ ]:
an_object.another_method()

#### Un use-case interessante delle metaclassi: 

classi che ammettono solo un <u>numero fissato</u> di oggetti

In [ ]:
class finiteobjects(type):
    _objects = []
    _n = 3
    def __call__(cls, *args, **kwargs):
        if len(cls._objects)<cls._n:
            obj = super(finiteobjects,cls).__call__(*args, **kwargs)
            cls._objects.append(obj)
            return obj

In [ ]:
class finiteobjectsclass(metaclass=finiteobjects):
    pass

In [ ]:
x  = finiteobjectsclass()

In [ ]:
y  = finiteobjectsclass()

In [ ]:
z  = finiteobjectsclass()

In [ ]:
x

In [ ]:
y

In [ ]:
z